In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

2023-04-22 02:47:58.057216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 02:47:58.181113: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [3]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation='swish', kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(100, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

2023-04-22 02:48:00.346695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-22 02:48:00.349643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-22 02:48:00.349962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-22 02:48:00.350605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

In [4]:
# 옵티마이저 및 학습률 스케줄러 정의
# 옵티마이저 정의
learning_rate = 1e-3

def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate


opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate )


model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [6]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR100", entity="hcim", name='LeNet-swish')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))


callbacks = [wandbCallback, LearningRateScheduler(learning_rate_schedule)]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Learning rate: 0.001
Epoch 1/200


2023-04-22 02:48:05.261743: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


387/391 [============================>.] - ETA: 0s - loss: 4.1370 - accuracy: 0.0696

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best)... Done. 0.0s


391/391 [==============================] - 11s 28ms/step - loss: 1.1529 - accuracy: 0.6714 - val_loss: 2.2104 - val_accuracy: 0.4830 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 84/200
391/391 [==============================] - 10s 25ms/step - loss: 1.1518 - accuracy: 0.6729 - val_loss: 2.2220 - val_accuracy: 0.4797 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 85/200
391/391 [==============================] - 9s 24ms/step - loss: 1.1518 - accuracy: 0.6734 - val_loss: 2.2165 - val_accuracy: 0.4803 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 86/200
391/391 [==============================] - 9s 23ms/step - loss: 1.1428 - accuracy: 0.6751 - val_loss: 2.2191 - val_accuracy: 0.4822 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 87/200
391/391 [==============================] - 9s 23ms/step - loss: 1.1408 - accuracy: 0.6780 - val_loss: 2.2294 - val_accuracy: 0.4819 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 88/200
391/391 [==============================] - 9s 24ms/step - loss: 1.1400 - accur

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230422_024802-8en9mdg3/files/model-best)... Done. 0.0s


391/391 [==============================] - 11s 28ms/step - loss: 1.1292 - accuracy: 0.6789 - val_loss: 2.2366 - val_accuracy: 0.4836 - lr: 1.0000e-04
Learning rate: 0.0001
Epoch 101/200
391/391 [==============================] - 9s 23ms/step - loss: 1.1264 - accuracy: 0.6789 - val_loss: 2.2377 - val_accuracy: 0.4810 - lr: 1.0000e-04
Learning rate: 1e-05
Epoch 102/200
391/391 [==============================] - 9s 23ms/step - loss: 1.1089 - accuracy: 0.6843 - val_loss: 2.2357 - val_accuracy: 0.4821 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 103/200
391/391 [==============================] - 9s 24ms/step - loss: 1.1100 - accuracy: 0.6844 - val_loss: 2.2353 - val_accuracy: 0.4825 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 104/200
391/391 [==============================] - 9s 24ms/step - loss: 1.1035 - accuracy: 0.6868 - val_loss: 2.2349 - val_accuracy: 0.4817 - lr: 1.0000e-05
Learning rate: 1e-05
Epoch 105/200
391/391 [==============================] - 11s 27ms/step - loss: 1.1016 - accu